In [1]:
import requests
import re
from bs4 import BeautifulSoup
import math
import datetime
import time
import random
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pandas as pd
import os


'''
지난 날의 검색어 순위 min분 단위로 가져와 엑셀에 저장 및 워드클라우드 출력
'''

def getKeywords(day):

    if type(day) != str:
        print('invalid type')
        return

    '''
    정규표현식으로 str이 맞는 형식 (yyyy-mm-dd) 인지 체크하는 코드
    '''
    match_day = re.compile('20\d{2}-[01]\d-[0123]\d')
    
    if match_day.match(day) == None:
        print('input : 20yy-mm-dd')
        return
    
    # 지난 날을 입력하지 않으면 종료
    min_day = '2017-03-29'
    today = datetime.datetime.today()
    ytd_str = '%4d-%02d-%02d' %(today.year, today.month, today.day-1)
    
    if ytd_str < day or min_day > day:
        print('input :', min_day, '~', ytd_str)
        return
    
    # 네이버가 유저가 아닌 다른 접속은 차단해 놓았기 때문에 header로 나 유저요~ 라고 해줌
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}

    # 몇 분 단위로 검색할지
    min = 30
    
    # url 뒤에 붙여줄 시간 문자열을 리스트로 (문자 형식 yyyy-mm-ddThh:mm:ss)
    times = [day+'T%02d:%02d:00' %(x, y) for x in range(0,24) for y in range(0,60) if y%min == 0]
    
    # 순위에 따른 점수 : x등은 log(22-x)점
    ranking_score = {x:math.log(22-x) for x in range(1, 21)}
    
    # 검색어와 총 점수 저장하는 공간 (워드클라우드)
    key_dic = dict()
    
    # 시간별 순위 저장할 공간 (엑셀용)
    rank_dic = dict()
    
    # sleep 주기
    sleep_times = 0
    
    for t in times:
        # 슬립 안해주면 네이버에서 ip밴당함
        if (sleep_times)%20 == 0:
            time.sleep(random.random()+1)
        
        '''
        - 기간별로 html 구조가 달라서 좀 더 조건에 맞춰서 코드를 만들어야 함 (만듬)
        '''
        
        url = 'https://datalab.naver.com/keyword/realtimeList.naver?datetime='
        req = requests.get(url+t, headers = headers)
        soup = BeautifulSoup(req.text, 'html.parser')
        
        
        # 최신 html 형식
        ranking_box = soup.find_all(class_ = 'ranking_item')
        
        # 최신 html형식으로 find 못했을 경우
        # 2018.10.10 ~ 2019.11.28 html 형식
        if len(ranking_box) < 2:
            all_age = soup.find('div', {'data-age':'all'})
            if all_age != None:
                ranking_box = all_age.find_all(class_='list')        
        
        # ... ~ 2019.11.28 find 안됬을 때
        # 2017.03.29 ~ 2018.10.10 html 형식
        if len(ranking_box) < 2:
            select_date = soup.find(class_ = 'keyword_rank select_date')
            if select_date != None:
                ranking_box = select_date.find_all(class_='list')
        
        # 시간 단위 검색어 순위 딕셔너리에 'hh:mm' 을 컬럼명으로 리스트 생성
        colm = re.sub('\d{2}(\d{2}-\d{2}-\d{2})T(\d{2}:\d{2}):\d{2}', '\\2', t)
        rank_dic[colm] = list()
        
        for i in ranking_box:
            num = int(i.find(class_=re.compile('.*num$')).text)
            title = i.find(class_=re.compile('.*title$')).text
            
#             print(num, title)
            
            #엑셀용 딕셔너리에 저장
            rank_dic[colm].append(title)
            
            # 워드클라우드용 딕셔너리에 저장
            if title in key_dic:
                key_dic[title] += ranking_score[num]
            else:
                key_dic[title] = ranking_score[num] 
        
#         print(colm, rank_dic[colm])
        sleep_times += 1
    
    if len(key_dic) > 0:
        keyword = {x:y for x, y in key_dic.items() if y>(400/min)} # 데이터 너무 많아서 줄여봄

        wordcloud = WordCloud(font_path = 'C:/Windows/Fonts/malgun.ttf', background_color='white',colormap = "Accent_r", width=800, height=800).generate_from_frequencies(keyword)
        fig = plt.figure(figsize=(20, 20))
        plt.imshow(wordcloud)
        plt.axis('off')
        #plt.show()
        fig.savefig(day+'.jpg')
    else:
        print('not exist key data')
    
    if len(rank_dic) > 0:
        df = pd.DataFrame.from_dict(rank_dic, orient='index', columns=[i for i in range(1,21)])
        writer = pd.ExcelWriter(day+'.xlsx', engine = 'xlsxwriter')
        df.to_excel(writer, sheet_name='Sheet1')
        writer.close()
    else:
        print('not exist rank data')
        
    return


In [2]:
'''
현재 시간 검색어순위 엑셀에 갱신
'''

def save_NaverRanking(n):
    filename1 = re.match('\w+\.xlsx', n)
    
    if (filename1 == None):
        print(n, '형식에 맞지 않습니다.')
        print('*.xlsl 형태로 입력 해주세요')
        return
    else: 
        xlsxname = filename1[0]
        print(xlsxname, '이 생성(갱신)되었습니다.')
    
    
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}
    url = 'https://datalab.naver.com/keyword/realtimeList.naver?datetime='

    secs = time.time()
    checktime = time.localtime(secs-60)
    timesample = time.strftime('%Y-%m-%d'+'T'+'%H:%M:%S', checktime)

    response = requests.get(url+timesample, headers = headers)
    rank_data = BeautifulSoup(response.text, 'html.parser')
    ranklist = []
    rank = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
    for ranking in rank_data.select("span.item_title"):
        ranklist.append(ranking.get_text())
    
    key_val = [rank, ranklist]
    rank_dict = dict(zip(*key_val))
    rank_data = pd.DataFrame(rank_dict, index = [timesample])
    
    if os.path.isfile(xlsxname) :
        rank_data_read = pd.read_excel(xlsxname, index_col=0)
        rank_data = pd.concat([rank_data_read, rank_data])
        rank_data.to_excel(xlsxname)

    else :
        rank_data.to_excel(xlsxname)

In [3]:
'''
두 함수 조합해서 쓰레드 만들어보기
1. 날짜가 바뀌면 전날의 워드크라우드 + 엑셀 파일 만드는 함수 실행
2. 5분 단위로 엑셀에 인기검색어 추가
'''
import threading
import time

class Update(threading.Thread):
    
    def __init__(self, name):
        threading.Thread.__init__(self)
        self.running = True
        self.name = name
        
    def run(self):
        lastUpdate = self.getDateStr()
        getKeywords(lastUpdate)
        print('update :', lastUpdate)
        lastMin = self.getMin()
        
        while self.running:
            date = self.getDateStr()
            
            if lastUpdate != date:
                getKeywords(date)
                lastUpdate = date
                print('update :', lastUpdate)
            
            
            if (lastMin != self.getMin()) and (self.getMin()%5 == 0):
                lastMin = self.getMin()
                save_NaverRanking(self.name)
                  
            time.sleep(5)
            
    
    def getDateStr(self):
        now = datetime.datetime.today()
        return '%d-%02d-%02d' %(now.year, now.month, now.day-1)
    
    def getMin(self):
        return datetime.datetime.today().minute
    
    def setRunning(self, running):
        self.running = running

update = Update('update_rank.xlsx')
update.start()

update : 2020-07-25
update : 2020-07-26
update_rank.xlsx 이 생성(갱신)되었습니다.
update_rank.xlsx 이 생성(갱신)되었습니다.
update_rank.xlsx 이 생성(갱신)되었습니다.
